In [1]:
import numpy as np
import pandas as pd
import math
import random
import pickle

# to make the experimens replicable
random.seed(123456)

In [2]:
df = pd.read_pickle('../data/atti-dirigenti-processed.pkl')

# Dataset Creation

for the analysis we are going to create a dataset where the `OGGETTO` is the independent variable while the `UFFICIO_DG` is the dependent variable.

In [3]:
dataset = df[['OGGETTO', 'UFFICIO_DG', 'DATA_ATTO']]
dataset.shape

(152455, 3)

### Groups the documents by office

In [4]:
documents_per_office = dataset.groupby(['UFFICIO_DG']).count()
documents_per_office.describe()

,OGGETTO,DATA_ATTO
count,36.000000,36.000000
mean,4234.861111,4234.861111
std,4010.579446,4010.579446
min,105.000000,105.000000
25%,1011.250000,1011.250000
50%,2870.000000,2870.000000
75%,7239.750000,7239.750000
max,13826.000000,13826.000000


###  Dataset Creation

Here, we can:
- select the documents with frequency greater that the 25 percentile
- or use all the dataset

In [5]:
value = 2000
sel_dataset = documents_per_office[documents_per_office.OGGETTO >= 2000]
sel_dataset.shape

(20, 2)

In [6]:
sel_dataset['UFFICIO_DG'] = sel_dataset.index

/home/fabio/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
sel_dataset.describe()

,OGGETTO,DATA_ATTO
count,20.000000,20.000000
mean,6902.150000,6902.150000
std,3539.235857,3539.235857
min,2624.000000,2624.000000
25%,3719.250000,3719.250000
50%,6590.500000,6590.500000
75%,9389.500000,9389.500000
max,13826.000000,13826.000000


In [8]:
sel_dataset.head()

,OGGETTO,DATA_ATTO,UFFICIO_DG
UFFICIO_DG,,,
01025,3030,3030,01025
01928,2710,2710,01928
01934,4923,4923,01934
01937,7160,7160,01937
01943,3833,3833,01943


### Join the dataset to select a subset of the data

In [9]:
final_ds = dataset.merge(sel_dataset, how='inner', on=['UFFICIO_DG'])

In [10]:
final_ds.shape

(138043, 5)

### Select the OGGETTO and UFFICIO

In [11]:
final_ds = final_ds[['OGGETTO_x',"UFFICIO_DG"]]
final_ds.head()

,OGGETTO_x,UFFICIO_DG
0,DGR 968/07 e s.m.i. Accreditamento degli organ...,DIREZIONE ISTRUZIONE E FORMAZIONE
1,Nomina Commissione d’esame per il percorso for...,DIREZIONE ISTRUZIONE E FORMAZIONE
2,Nomina della Commissione d'esame matricola 201...,DIREZIONE ISTRUZIONE E FORMAZIONE
3,Progetti formativi per drop-out a.s.f. 2015-20...,DIREZIONE ISTRUZIONE E FORMAZIONE
4,REG (CE) 1080/2006-Por Creo Fesr 2007-2013-Lin...,DIREZIONE ISTRUZIONE E FORMAZIONE


In [12]:
len(set(final_ds['UFFICIO_DG']))

20

### Transform it in a dataset

In [13]:
samples = []
labels = []

In [14]:
for text, label in final_ds.as_matrix():
    samples.append(text)
    labels.append(label)

In [15]:
samples[:5]

["DGR 968/07 e s.m.i. Accreditamento degli organismi formativi. Rilascio dell'accreditamento all'organismo formativo Bioscience Research Center - cod. GR1035.",
 'Nomina Commissione d’esame per il percorso formativo “Formazione obbligatoria per utilizzatori professionali di prodotti fitosanitari”, MATRICOLA N. 2016SI0046',
 "Nomina della Commissione d'esame matricola 2016PI0301. Agenzia Formativa Cescot",
 'Progetti formativi per drop-out a.s.f. 2015-2016. Integrazione impegno per progetto "Figaro - Operatore del benessere (acconciatura)"',
 'REG (CE) 1080/2006-Por Creo Fesr 2007-2013-Linea di intevento 5.1.d-Chiusura attività']

In [16]:
labels[:5]

['DIREZIONE ISTRUZIONE E FORMAZIONE',
 'DIREZIONE ISTRUZIONE E FORMAZIONE',
 'DIREZIONE ISTRUZIONE E FORMAZIONE',
 'DIREZIONE ISTRUZIONE E FORMAZIONE',
 'DIREZIONE ISTRUZIONE E FORMAZIONE']

In [17]:
samples = np.array(samples)
labels = np.array(labels)

In [18]:
with open('../data/dataset-dirigenti.pkl', 'wb') as o:
    pickle.dump((samples, labels), o)

In [19]:
dataset_path = '../data/dataset-dirigenti.pkl'

with open(dataset_path, 'rb') as f:
    samples, labels = pickle.load(f)

## Create Train and Test Set

- shuffle data
- split the dataset into 80-20

In [20]:
from sklearn.model_selection import StratifiedShuffleSplit

In [21]:
split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

In [22]:
for train, test in split_train_test.split(samples, labels):
    train_samples, test_samples = samples[train], samples[test]
    train_labels, test_labels = labels[train], labels[test]

Split train into train and validation set

In [23]:
split_train_val = StratifiedShuffleSplit(1,test_size=0.1, random_state=123456)

In [24]:
for train, val in split_train_val.split(train_samples, train_labels):
    train_samples_, val_samples = train_samples[train], train_samples[val]
    train_labels_, val_labels = train_labels[train], train_labels[val]

In [25]:
print('labels training {}'.format(len(set(train_labels_))))
print('labels validation {}'.format(len(set(val_labels))))
print('labels test {}'.format(len(set(test_labels))))

labels training 20
labels validation 20
labels test 20


In [26]:
print('samples training {}'.format(len(set(train_samples_))))
print('samples validation {}'.format(len(set(val_samples))))
print('samples test {}'.format(len(set(test_samples))))

samples training 96802
samples validation 10944
samples test 27216


In [27]:
len(samples)

138043

In [28]:
with open('../data/dataset-dirigenti-split.pkl', 'wb') as o:
    pickle.dump((train_samples_, train_labels_, val_samples, val_labels, test_samples, test_labels), o)